### GOOGLE - GEMINI

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
from IPython.display import display, Markdown

load_dotenv(find_dotenv())

import google.generativeai as gemini_AI

GEMININAI_API_KEY = os.getenv("GEMININ_API_KEY")

gemini_AI.configure(api_key=GEMININAI_API_KEY)

model = gemini_AI.GenerativeModel('gemini-pro')

response = model.generate_content("Hi how are you?")

display(Markdown(response.text))


### OPENAI

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
from IPython.display import display, Markdown
import asyncio
import openai

load_dotenv(find_dotenv())

client = OpenAI()


response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    response_format={"type": "text"},
    # response_format={"type": "json_object"},
    messages=[
        {
            "role": "system",
            # "content": "helpful assisstant that produces output in JSON",
            "content": "helpful assisstant",
        },
        {
            "role": "user",
            "content": "How are you?"
        },
    ],
)

# display(Markdown(response.choices[0].message))
response.choices[0].message

### LLAMA-2

##### HUGGINGFACE MISTRAL-AI

In [ ]:
import requests
from dotenv import load_dotenv, find_dotenv
import os
from IPython.display import display, Markdown

API_URL = (
    "https://api-inference.huggingface.co/models/mistralai/Mixtral-8x7B-Instruct-v0.1"
)

API_TOKEN = os.getenv("HUGGINGFACE_API_KEY")

if API_TOKEN:
    headers = {"Authorization": f"Bearer {API_TOKEN}"}

    def query(payload):
        response = requests.post(API_URL, headers=headers, json=payload)
        return response.json()

    output = query(
        {
            "inputs": "How are you?",
        }
    )

    display(output)
else:
    print("Please set API_TOKEN to query the API.")

In [ ]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'meta-llama/Llama-2-70b-chat-hf',
	'SM_NUM_GPUS': json.dumps(8),
	'HUGGING_FACE_HUB_TOKEN': os.getenv("HUGGINGFACE_API_KEY"),
}

assert hub['HUGGING_FACE_HUB_TOKEN'] != os.getenv("HUGGINGFACE_API_KEY"), "You have to provide a token."

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="1.1.0"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.48xlarge",
	container_startup_health_check_timeout=1200,
  )
  
# send request
predictor.predict({
	"inputs": "My name is Julien and I like to",
})

In [ ]:
import fireworks.client
from dotenv import load_dotenv
import os
load_dotenv()

fireworks.client.api_key = os.getenv("FIREWORKS_API_KEY")
completion = fireworks.client.ChatCompletion.create(
  model="accounts/fireworks/models/llama-v2-7b-chat",
  messages=[
    {
      "role": "user",
      "content": "hi how are you?",
    }
  ],
  # stream=True,
  n=1,
  max_tokens=1000,
  temperature=0.1,
  top_p=0.9, 
  stop=[],
)

# for item in completion:
#     print(item)
# completion_list = list(completion)
# for response in completion_list:
#     for choice in response.choices:
#         print(choice.delta.content)

# completion_list = list(completion)
# print(completion_list[1].choices[0].delta.content)
# completion_list
completion.choices[0].message.content